In [ ]:
# !pip3 install catboost
# !pip3 install xgboost

In [1]:
# !pip3 install lightgbm
# !pip3 install steppy

In [1]:
import logging
import os
import random
import sys
import multiprocessing as mp
from functools import reduce

import six
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
import yaml
from attrdict import AttrDict
# from steppy.base import BaseTransformer
import joblib
import sklearn.preprocessing as prep

In [2]:
import os
import sys
from functools import partial
import warnings
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score as RSquared
from src.utils import parallel_apply, last_k_bureau_features, add_features_in_group
# from src.feature_extraction import add_features_in_group
from functools import partial
from tqdm.notebook import tqdm
import multiprocessing as mp

In [3]:
DIR='/Users/melo/pyprogs/open-solution-home-credit-master/home-credit-default-risk'
df = pd.read_csv(os.path.join(DIR, 'application_train.csv'))
bureau = pd.read_csv(os.path.join(DIR, 'bureau.csv'))

In [4]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [9]:
### bureau EDA
bureau['bureau_credit_type_consumer'] = (bureau['CREDIT_TYPE'] == 'Consumer credit').astype(int)
bureau['bureau_credit_type_car'] = (bureau['CREDIT_TYPE'] == 'Car loan').astype(int)
bureau['bureau_credit_type_mortgage'] = (bureau['CREDIT_TYPE'] == 'Mortgage').astype(int)
bureau['bureau_credit_type_credit_card'] = (bureau['CREDIT_TYPE'] == 'Credit card').astype(int)
bureau['bureau_credit_type_other'] = (~(bureau['CREDIT_TYPE'].isin(['Consumer credit',
                                                        'Car loan', 'Mortgage', 'Credit card']))).astype(int)
bureau['bureau_unusual_currency'] = (~(bureau['CREDIT_CURRENCY'] == 'currency 1')).astype(int)

In [10]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,bureau_credit_type_consumer,bureau_credit_type_car,bureau_credit_type_mortgage,bureau_credit_type_credit_card,bureau_credit_type_other,bureau_unusual_currency
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,...,0.0,Consumer credit,-131,NaN,1,0,0,0,0,0
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,...,0.0,Credit card,-20,NaN,0,0,0,1,0,0
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,...,0.0,Consumer credit,-16,NaN,1,0,0,0,0,0
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,...,0.0,Credit card,-16,NaN,0,0,0,1,0,0
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,...,0.0,Consumer credit,-21,NaN,1,0,0,0,0,0


In [11]:
features = pd.DataFrame({'SK_ID_CURR':bureau['SK_ID_CURR'].unique()})
features.head()

,SK_ID_CURR
0,215354
1,162297
2,402440
3,238881
4,222183


In [12]:
groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])
group_object = groupby_SK_ID_CURR[['bureau_credit_type_consumer',
                                   'bureau_credit_type_car',
                                   'bureau_credit_type_mortgage',
                                   'bureau_credit_type_credit_card',
                                   'bureau_credit_type_other',
                                   'bureau_unusual_currency']].agg(['sum', 'mean']).reset_index()

group_object.columns = ['{}_{}'.format(a,b) for a,b in group_object.columns.tolist()]
group_object.rename(index=str, columns={'SK_ID_CURR_': 'SK_ID_CURR'},inplace=True)
features = features.merge(group_object, on=['SK_ID_CURR'], how='left')


In [13]:
bureau.sort_values(['SK_ID_CURR','DAYS_CREDIT'],ascending=False, inplace=True)
bureau['days_credit_diff'] = bureau['DAYS_CREDIT'].diff().replace(np.nan, 0)    

bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)

groupby_SK_ID_CURR = bureau.groupby(by=['SK_ID_CURR'])

In [14]:
group_object = groupby_SK_ID_CURR['bureau_credit_active_binary'].agg('sum').reset_index()
group_object.rename(index=str, columns={'bureau_credit_active_count': 'bureau_credit_active_binary'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

In [15]:
group_object = groupby_SK_ID_CURR['SK_ID_BUREAU'].agg('nunique').reset_index()
group_object.rename(index=str, columns={'SK_ID_BUREAU': 'bureau_query_count'},inplace=True)

features = features.merge(group_object, on=['SK_ID_CURR'], how='left')

In [27]:
# g['SK_ID_CURR'] = g['SK_ID_CURR'].map(lambda x: x.lstrip('(').rstrip(',)'))
# g['SK_ID_CURR'] = g['SK_ID_CURR'].map(lambda x: x[0])

In [23]:
# g[g['SK_ID_CURR']==(100001,)]

,SK_ID_CURR,last_60_days_credit_diff_sum,last_60_days_credit_diff_min,last_60_days_credit_diff_max,last_60_days_credit_diff_median,last_60_days_credit_diff_std,last_60_CNT_CREDIT_PROLONG_sum,last_60_CNT_CREDIT_PROLONG_std,last_60_bureau_credit_active_binary_sum,last_60_bureau_credit_active_binary_mean,...,all_records_bureau_credit_active_binary_sum,all_records_bureau_credit_active_binary_mean,all_records_bureau_credit_type_consumer_sum,all_records_bureau_credit_type_consumer_mean,all_records_bureau_credit_type_car_sum,all_records_bureau_credit_type_car_mean,all_records_bureau_credit_type_credit_card_sum,all_records_bureau_credit_type_mortgage_sum,all_records_bureau_credit_type_other_sum,all_records_bureau_credit_type_other_mean
0,"(100001,)",1388.0,1388.0,1388.0,1388.0,NaN,0,NaN,1,1.0,...,3,0.428571,7,1.0,0,0.0,0,0,0,0.0


In [26]:
# type(g['SK_ID_CURR'])

pandas.core.series.Series

In [28]:
# g.head()

,SK_ID_CURR,last_60_days_credit_diff_sum,last_60_days_credit_diff_min,last_60_days_credit_diff_max,last_60_days_credit_diff_median,last_60_days_credit_diff_std,last_60_CNT_CREDIT_PROLONG_sum,last_60_CNT_CREDIT_PROLONG_std,last_60_bureau_credit_active_binary_sum,last_60_bureau_credit_active_binary_mean,...,all_records_bureau_credit_active_binary_sum,all_records_bureau_credit_active_binary_mean,all_records_bureau_credit_type_consumer_sum,all_records_bureau_credit_type_consumer_mean,all_records_bureau_credit_type_car_sum,all_records_bureau_credit_type_car_mean,all_records_bureau_credit_type_credit_card_sum,all_records_bureau_credit_type_mortgage_sum,all_records_bureau_credit_type_other_sum,all_records_bureau_credit_type_other_mean
0,100001,1388.0,1388.0,1388.0,1388.0,NaN,0,NaN,1,1.0,...,3,0.428571,7,1.000000,0,0.0,0,0,0,0.0
1,100002,0.0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,...,2,0.250000,4,0.500000,0,0.0,4,0,0,0.0
2,100003,0.0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,...,1,0.250000,2,0.500000,0,0.0,2,0,0,0.0
3,100004,0.0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,...,0,0.000000,2,1.000000,0,0.0,0,0,0,0.0
4,100005,0.0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,...,2,0.666667,2,0.666667,0,0.0,1,0,0,0.0


In [19]:
# features.head()

,SK_ID_CURR,bureau_credit_type_consumer_sum,bureau_credit_type_consumer_mean,bureau_credit_type_car_sum,bureau_credit_type_car_mean,bureau_credit_type_mortgage_sum,bureau_credit_type_mortgage_mean,bureau_credit_type_credit_card_sum,bureau_credit_type_credit_card_mean,bureau_credit_type_other_sum,bureau_credit_type_other_mean,bureau_unusual_currency_sum,bureau_unusual_currency_mean,bureau_credit_active_binary,bureau_query_count
0,215354,7,0.636364,1,0.090909,0,0.000000,3,0.272727,0,0.0,0,0.0,6,11
1,162297,3,0.500000,0,0.000000,1,0.166667,2,0.333333,0,0.0,0,0.0,3,6
2,402440,1,1.000000,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.0,1,1
3,238881,5,0.625000,0,0.000000,0,0.000000,3,0.375000,0,0.0,0,0.0,3,8
4,222183,4,0.500000,1,0.125000,0,0.000000,3,0.375000,0,0.0,0,0.0,5,8


In [20]:
# features[features['SK_ID_CURR']==100001]

,SK_ID_CURR,bureau_credit_type_consumer_sum,bureau_credit_type_consumer_mean,bureau_credit_type_car_sum,bureau_credit_type_car_mean,bureau_credit_type_mortgage_sum,bureau_credit_type_mortgage_mean,bureau_credit_type_credit_card_sum,bureau_credit_type_credit_card_mean,bureau_credit_type_other_sum,bureau_credit_type_other_mean,bureau_unusual_currency_sum,bureau_unusual_currency_mean,bureau_credit_active_binary,bureau_query_count
57591,100001,7,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,3,7


In [21]:
# groupby_SK_ID_CURR.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,AMT_ANNUITY,bureau_credit_type_consumer,bureau_credit_type_car,bureau_credit_type_mortgage,bureau_credit_type_credit_card,bureau_credit_type_other,bureau_unusual_currency,days_credit_diff,bureau_credit_active_binary,bureau_credit_enddate_binary
1010009,456255,5126333,Active,currency 1,-363,0,1463.0,NaN,0.000,0,...,0.0,1,0,0,0,0,0,0.0,1,1
1010008,456255,5126332,Active,currency 1,-378,0,27320.0,NaN,NaN,0,...,NaN,0,0,0,1,0,0,-15.0,1,1
1010010,456255,5126334,Active,currency 1,-451,0,279.0,NaN,15439.905,0,...,3244.5,1,0,0,0,0,0,-73.0,1,1
1010005,456255,5126329,Active,currency 1,-573,0,523.0,NaN,NaN,0,...,0.0,1,0,0,0,0,0,-122.0,1,1
1010004,456255,5126328,Closed,currency 1,-970,0,-621.0,-712.0,NaN,0,...,NaN,1,0,0,0,0,0,-397.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248489,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,...,10822.5,1,0,0,0,0,0,1388.0,1,1
248490,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,...,9364.5,1,0,0,0,0,0,-271.0,1,1
248488,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,...,4630.5,1,0,0,0,0,0,-239.0,1,1
248484,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,...,0.0,1,0,0,0,0,0,-298.0,0,0


In [31]:
func = partial(last_k_bureau_features, periods=[60, 180, 360, 720, 1e25])

g = parallel_apply(groupby_SK_ID_CURR, func, index_name='SK_ID_CURR',
                   num_workers=10, chunk_size=10000).reset_index()
# not sure why the results in g , the ID field is a tuple. updating it to get the ID as an int
g['SK_ID_CURR'] = g['SK_ID_CURR'].map(lambda x: x[0])
features = features.merge(g, on='SK_ID_CURR', how='left')

100%|█████████████████████████████████████████████████████████████████████████████████| 31/31.0 [03:36<00:00,  6.97s/it]


In [29]:
# features = features.merge(g, on='SK_ID_CURR', how='left')

In [30]:
features.head()

,SK_ID_CURR,bureau_credit_type_consumer_sum,bureau_credit_type_consumer_mean,bureau_credit_type_car_sum,bureau_credit_type_car_mean,bureau_credit_type_mortgage_sum,bureau_credit_type_mortgage_mean,bureau_credit_type_credit_card_sum,bureau_credit_type_credit_card_mean,bureau_credit_type_other_sum,...,all_records_bureau_credit_active_binary_sum,all_records_bureau_credit_active_binary_mean,all_records_bureau_credit_type_consumer_sum,all_records_bureau_credit_type_consumer_mean,all_records_bureau_credit_type_car_sum,all_records_bureau_credit_type_car_mean,all_records_bureau_credit_type_credit_card_sum,all_records_bureau_credit_type_mortgage_sum,all_records_bureau_credit_type_other_sum,all_records_bureau_credit_type_other_mean
0,215354,7,0.636364,1,0.090909,0,0.000000,3,0.272727,0,...,6,0.545455,7,0.636364,1,0.090909,3,0,0,0.0
1,162297,3,0.500000,0,0.000000,1,0.166667,2,0.333333,0,...,3,0.500000,3,0.500000,0,0.000000,2,1,0,0.0
2,402440,1,1.000000,0,0.000000,0,0.000000,0,0.000000,0,...,1,1.000000,1,1.000000,0,0.000000,0,0,0,0.0
3,238881,5,0.625000,0,0.000000,0,0.000000,3,0.375000,0,...,3,0.375000,5,0.625000,0,0.000000,3,0,0,0.0
4,222183,4,0.500000,1,0.125000,0,0.000000,3,0.375000,0,...,5,0.625000,4,0.500000,1,0.125000,3,0,0,0.0


In [33]:
df1 = df.merge(features,
                      left_on=['SK_ID_CURR'],
                      right_on=['SK_ID_CURR'],
                      how='left',
                      validate='one_to_one')

In [35]:
df1.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,all_records_bureau_credit_active_binary_sum_y,all_records_bureau_credit_active_binary_mean_y,all_records_bureau_credit_type_consumer_sum_y,all_records_bureau_credit_type_consumer_mean_y,all_records_bureau_credit_type_car_sum_y,all_records_bureau_credit_type_car_mean_y,all_records_bureau_credit_type_credit_card_sum_y,all_records_bureau_credit_type_mortgage_sum_y,all_records_bureau_credit_type_other_sum_y,all_records_bureau_credit_type_other_mean_y
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,2.0,0.25,4.0,0.5,0.0,0.0,4.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.0,0.25,2.0,0.5,0.0,0.0,2.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,0.00,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,0.00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
X = df1.drop(columns='TARGET')
y = df1['TARGET']

#manage the category value fields
obj_feat = list(X.loc[:, X.dtypes == 'object'].columns.values)

for feature in obj_feat:
    X[feature] = pd.Series(X[feature], dtype="category")

In [40]:
X['NAME_HOUSING_TYPE'].head()

0    House / apartment
1    House / apartment
2    House / apartment
3    House / apartment
4    House / apartment
Name: NAME_HOUSING_TYPE, dtype: category
Categories (6, object): ['Co-op apartment', 'House / apartment', 'Municipal apartment', 'Office apartment', 'Rented apartment', 'With parents']

In [41]:
import lightgbm


start = time.time()

lgbmr = lightgbm.LGBMRegressor()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=1)

lgbmr.fit(X_train, y_train)
y_pred = lgbmr.predict(X_test)

lgbm_rsquared = np.sqrt(RSquared(y_test, y_pred))
print()
print("R squared for LightGBM: ", np.mean(lgbm_rsquared))

end = time.time()
diff = end - start
print('Execution time:', diff)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074321 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28320
[LightGBM] [Info] Number of data points in the train set: 246008, number of used features: 301
[LightGBM] [Info] Start training from score 0.080709

R squared for LightGBM:  0.30051326129932343
Execution time: 4.783046007156372


In [ ]:
# from catboost import CatBoostRegressor, Pool
# # X = df.drop(columns='Life expectancy')
# # y = df['Life expectancy']


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
#                                                     random_state=1)

# pool_train = Pool(X_train, y_train,
#                   cat_features = ['Country','Year','Status'])

# pool_test = Pool(X_test, cat_features = ['Country','Year','Status'])


# #CatBoost



# start = time.time()

# cbr = CatBoostRegressor(iterations=100,max_depth=2)

# cbr.fit(pool_train)
# y_pred = cbr.predict(X_test)




# cb_rsquared = np.sqrt(RSquared(y_test, y_pred))
# print("R Squared for CatBoost: ", np.mean(cb_rsquared))

# end = time.time()
# diff = end - start
# print('Execution time:', diff)


In [ ]:
# #XGBoost
# import xgboost as xgb
# from sklearn import preprocessing
# X = df.drop(columns='Life expectancy')
# y = df['Life expectancy']

# lbl = preprocessing.LabelEncoder()
# #Country','Year','Status
# X['Country'] = lbl.fit_transform(X['Country'].astype(str))
# X['Year'] = lbl.fit_transform(X['Year'].astype(str))
# X['Status'] = lbl.fit_transform(X['Status'].astype(str))

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
#                                                     random_state=1)

# start = time.time()
# #X_train["Species"].astype("category")
# xgbr = xgb.XGBRegressor()

# xgbr.fit(X_train, y_train)
# y_pred = xgbr.predict(X_test)

# xgb_rmse = np.sqrt(RSquared(y_test, y_pred))
# print("R Squared for XGBoost: ", np.mean(xgb_rmse))

# end = time.time()
# diff = end - start
# print('Execution time:', diff)

# 1. Community support/Performance/Want to tune - XGBOOST
# 2. Hurry to train/Performance/Dont want to tune - LightGBM
# 3. More categorical, GPU, Large data - CatBoost